In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
sudoku = cv2.imread('data/sudoku_1.jpg')
sudoku = cv2.resize(sudoku,(282,282))
# plt.figure()
# plt.imshow(sudoku)
# plt.show()

In [3]:
gray = cv2.cvtColor(sudoku, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 1)
thresh = cv2.adaptiveThreshold(blur, maxValue=255, adaptiveMethod=1, thresholdType=1, blockSize=11, C=2)

In [4]:
# plt.title('Gaussian Blur')
# plt.imshow(blur)
# plt.show()
# plt.title('Adaptive Threshold')
# plt.imshow(thresh)
# plt.show()

In [5]:
sudoku_contour = sudoku.copy()
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
sudoku_contour = cv2.drawContours(sudoku_contour, contours[1], -1, (0, 255, 0), 3)


In [6]:
# plt.title('Contours')
# plt.imshow(sudoku_contour)
# plt.show()

In [7]:
def bigContour(contours):
    big = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50: #if too small, will find noise
            perimeter = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*perimeter, True)
            if area > max_area and len(approx)==4: #checking of rect/square
                big = approx
                max_area = area
    return big, max_area

In [8]:
# order of the 4 points should be the same always. reorder() ensures this
def reorder(points):
    points = points.reshape((4,2))
    points_new = np.zeros((4,1,2), dtype=np.int32)
    add = points.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] =  points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]
    return points_new

In [9]:
sudoku_big_contour = sudoku.copy()
big, max_area = bigContour(contours[1])
if big.size!=0:
    big = reorder(big)
    sudoku_big_contour = cv2.drawContours(sudoku_big_contour, big, -1, (255,255,0), 20) #draw the biggest contour
    # preapres points for warp
    pts1 = np.float32(big) 
    pts2 = np.float32([[0,0], [252,0], [0,252], [252, 252]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    warped_sudoku = cv2.warpPerspective(sudoku, matrix, (252, 252))
    warped_sudoku = cv2.cvtColor(warped_sudoku, cv2.COLOR_RGB2GRAY)

In [ ]:
cv2.imshow('warped', warped_sudoku)
cv2.waitKey(0)

In [ ]:
# plt.title('Big Contour')
# plt.imshow(sudoku_big_contour)
# plt.title('Warped')
# plt.show()
# plt.imshow(warped_sudoku)
# plt.show()

In [ ]:
def split_boxes(image):
    rows = np.vsplit(image, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

In [ ]:
# detected = sudoku.copy()


In [ ]:
warped_sudoku

In [ ]:
for i,img in enumerate(warped_sudoku):
    mean = np.mean(img)
    std = np.std(img)
    warped_sudoku[i] = (img-mean)/std
      

In [ ]:
boxes = split_boxes(warped_sudoku)
for i in range(len(boxes)):
    for j in range(len(boxes[i])):
        boxes[i][j][0]=0
        boxes[i][j][-1]=0

In [ ]:
nums = np.array([1 if np.sum(boxes[i].flatten())/(255*81)>0.5 else 0 for i in range(len(boxes))])

In [ ]:
nums

In [ ]:
np.sum(boxes[1].flatten())/(255*81)

In [ ]:
plt.title('Sample')
plt.imshow(warped_sudoku)
plt.show()
plt.title('Box')
plt.imshow(boxes[4])
plt.show()

In [ ]:
np.shape(boxes)

In [ ]:
# from tensorflow import keras
from keras.models import load_model
model = load_model('model.h5')
boxes = np.reshape(boxes, (-1,28,28,1))
predicted_numbers = model.predict_classes(boxes)

In [ ]:
print(np.reshape(predicted_numbers*np.array(nums),(9,9)))
plt.show()
plt.imshow(sudoku)
plt.show()

In [ ]:
np.argsort(model.predict(boxes)[4])

In [ ]:
np.sort(model.predict(boxes)[4])